In [1]:
import pymysql
import csv
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

### 1. CSV 파일 MYSQL에 업로드 (초기 데이터 or 백업된 데이터)

#### Pandas와 MYSQL로 연동

In [25]:
engine = create_engine("mysql+mysqldb://root:" + "qhans7810!"
                       + "@192.168.0.2/covid", encoding = 'utf-8')

conn = engine.connect()

Exception during reset or similar
Traceback (most recent call last):
  File "/home/vanillapapaya/anaconda3/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "/home/vanillapapaya/anaconda3/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 867, in _reset
    pool._dialect.do_rollback(self)
  File "/home/vanillapapaya/anaconda3/lib/python3.7/site-packages/sqlalchemy/dialects/mysql/base.py", line 2253, in do_rollback
    dbapi_connection.rollback()
MySQLdb._exceptions.OperationalError: (2006, 'MySQL server has gone away')


#### 각 csv 파일 pandas로 업로드

In [26]:
domestic_sid = pd.read_csv("domestic_sid_edit.csv")
domestic_sid

,Date,Type,Total,Seoul,Busan,Daegu,Incheon,Gwangju,Daejeon,Ulsan,...,Gyeonggi,Gangwon,Chungbuk,Chungnam,Jeonbuk,Jeonnam,Gyeongbuk,Gyeongnam,Jeju,Quarantine
0,2020-02-24,under quarantine,763,30,17,443,2,9,3,1,...,35,6,3,1,3,1,186,20,2,0
1,2020-02-25,under quarantine,893,36,38,499,2,9,3,2,...,40,6,3,1,3,2,225,21,2,0
2,2020-02-26,under quarantine,1146,45,50,677,3,9,3,3,...,43,6,5,2,3,1,268,25,2,0
3,2020-02-27,under quarantine,1595,55,58,1017,3,9,8,6,...,55,6,7,7,3,1,321,36,2,0
4,2020-02-28,under quarantine,2022,62,63,1314,4,9,13,11,...,66,6,9,16,5,1,394,46,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2020-04-16,end quarantine,7757,250,111,5586,51,22,25,34,...,330,27,34,115,9,7,1044,82,5,4
132,2020-04-16,death,229,2,3,158,0,0,0,1,...,14,1,0,0,0,0,50,0,0,0
133,2020-04-17,under quarantine,2576,351,15,1042,38,8,15,7,...,296,23,8,24,8,8,269,31,8,403
134,2020-04-17,end quarantine,7829,268,112,5626,51,22,25,34,...,340,28,37,115,9,7,1037,84,5,5


- 특정 칼럼의 전일 대비 변화량을 계산하여 추가하는 Function

In [18]:
def new_column(df, column):
    pre_column = []
    for i in range(1, df[column].size):
        pre_column.append(df.loc[i][column])

    con_column = df[column].tolist()

    con_column.pop
    temp = []
    for i in range(0, len(pre_column)):
        temp.append(pre_column[i] - con_column[i])

    temp.insert(0, 0)
    newcolumn = "New" + column 
    df[newcolumn] = temp
    df

In [27]:
domestic_check = pd.read_csv("domestic_check_edit.csv")

#new_column(domestic_check, "Positive")
#new_column(domestic_check, "EndQuarantine")
#domestic_check.to_csv("domestic_check_edit.csv")
domestic_check

,Date,Positive,EndQuarantine,UnderQuarantine,Death,UnderExamination,Negative,Total,NewPositive,NewEndQuarantine
0,2020-01-21,1,0,1,0,3,7,11,0,0
1,2020-01-24,2,0,2,0,0,25,27,1,0
2,2020-01-26,3,0,3,0,1,47,51,1,0
3,2020-01-27,4,0,4,0,1,56,61,1,0
4,2020-01-30,6,0,6,0,41,199,246,2,0
...,...,...,...,...,...,...,...,...,...,...
77,2020-04-13,10537,7447,2873,217,13391,494815,518743,25,79
78,2020-04-14,10564,7534,2808,222,14651,502223,527438,27,87
79,2020-04-15,10591,7616,2750,225,15026,508935,534552,27,82
80,2020-04-16,10613,7757,2627,229,14268,513894,538775,22,141


In [28]:
domestic_group = pd.read_csv("domestic_group.csv")
domestic_group

,Date,Location,Total,ForeignInflow,GroupTotal,Shincheonji,GroupCases,PersonsInContact,ForeignGroupInflow,Other,New
0,2020-03-25,Seoul,347,58,259,7,194,35,23,30,13
1,2020-03-25,Busan,112,2,76,11,49,16,0,34,1
2,2020-03-25,Daegu,6456,2,5427,4387,321,719,0,1027,14
3,2020-03-25,Incheon,42,6,34,2,27,3,2,2,1
4,2020-03-25,Gwangju,19,4,14,9,0,2,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...
427,2020-04-17,Jeonnam,15,7,7,1,3,2,1,1,0
428,2020-04-17,Gyeongbuk,1356,9,1213,566,455,191,1,134,4
429,2020-04-17,Gyeongnam,115,14,84,32,45,6,1,17,0
430,2020-04-17,Jeju,13,8,1,0,0,0,1,4,0


#### pandas로 불러온 csv 파일을 MYSQL에 업로드 (replace)

In [29]:
domestic_sid.to_sql(name = "domestic_sid", con = engine, if_exists = 'replace')
domestic_check.to_sql(name = "domestic_check", con = engine, if_exists = 'replace')
domestic_group.to_sql(name = "domestic_group", con = engine, if_exists = 'replace')

### 2. PYTHON과 MYSQL 연동

In [108]:
conn = pymysql.connect(host = '192.168.0.2', user = 'vanillapapaya',
                      password = 'qhans7810!', db = 'covid',
                      charset = 'utf8')
curs = conn.cursor()

#### domestic_sid에 column 추가

In [30]:
# tab(공백)을 , 으로 변경하여 query 쉽게 짤 수 있도록
string = input()
string.replace(",", "").replace("	", ", ")

''

In [110]:
query = """

        INSERT INTO domestic_sid(Date, Type, Total, Seoul, Busan, Daegu, Incheon, Gwangju, Daejeon,
                               Ulsan, Sejong, Gyeonggi, Gangwon, Chungbuk, Chungnam, Jeonbuk, Jeonnam,
                               Gyeongbuk, Gyeongnam, Jeju, Quarantine)
        VALUES("2020-04-13", "under quarantine",
               2873, 394, 20, 1277, 48, 8, 16, 7, 24, 313, 20, 14, 30, 9, 9, 267, 31, 7, 379
            ),
            ("2020-04-13", "end quarantine",
               7447, 214, 103, 5395, 39, 19, 23, 33, 22, 305, 28, 31, 109, 8, 6, 1020, 84, 5, 3
            ),
            ("2020-04-13", "death",
               217, 2, 3, 147, 0, 0, 0, 1, 0, 13, 1, 0, 0, 0, 0, 50, 0, 0, 0
            )


"""

curs.execute(query)

3

#### domestic_check에 column 추가

In [109]:
query = """

        INSERT INTO domestic_check(Date, Total, Positive, EndQuarantine, UnderQuarantine, Death,
                               UnderExamination, Negative, NewPositive, NewEndQuarantine)
        VALUES("2020-04-14",
               527438, 10564, 7534, 2808, 222, 13391, 494815, 25, 79
            ) 


"""

curs.execute(query)

1

#### domestic_group에 column 추가
csv 파일 업로드로 대체

In [88]:
engine = create_engine("mysql+mysqldb://root:" + "qhans7810!"
                       + "@192.168.0.2/covid", encoding = 'utf-8')

conn = engine.connect()

domestic_group = pd.read_csv("domestic_group.csv")

domestic_group.to_(name = "domestic_group", con = engine, if_exists = 'replace')

Date	Location	Total	ForeignInflow	GroupTotal	Shincheonji	GroupCases	PersonsInContact	ForeignGroupInflow	Other	New


'Date, Location, Total, ForeignInflow, GroupTotal, Shincheonji, GroupCases, PersonsInContact, ForeignGroupInflow, Other, New'

In [112]:
query = """

    SELECT *
        FROM domestic_sid

"""

df = pd.read_sql(query, conn)
df
#df.to_csv("domestic_sid_edit.csv")

,index,Date,Type,Total,Seoul,Busan,Daegu,Incheon,Gwangju,Daejeon,...,Gyeonggi,Gangwon,Chungbuk,Chungnam,Jeonbuk,Jeonnam,Gyeongbuk,Gyeongnam,Jeju,Quarantine
0,0.0,2020-02-24,under quarantine,763,30,17,443,2,9,3,...,35,6,3,1,3,1,186,20,2,0
1,1.0,2020-02-25,under quarantine,893,36,38,499,2,9,3,...,40,6,3,1,3,2,225,21,2,0
2,2.0,2020-02-26,under quarantine,1146,45,50,677,3,9,3,...,43,6,5,2,3,1,268,25,2,0
3,3.0,2020-02-27,under quarantine,1595,55,58,1017,3,9,8,...,55,6,7,7,3,1,321,36,2,0
4,4.0,2020-02-28,under quarantine,2022,62,63,1314,4,9,13,...,66,6,9,16,5,1,394,46,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,119.0,2020-04-12,end quarantine,7368,202,103,5356,38,19,23,...,292,27,29,108,8,5,1013,84,4,3
120,120.0,2020-04-12,death,214,2,3,146,0,0,0,...,12,1,0,0,0,0,49,0,0,0
121,NaN,2020-04-13,under quarantine,2873,394,20,1277,48,8,16,...,313,20,14,30,9,9,267,31,7,379
122,NaN,2020-04-13,end quarantine,7447,214,103,5395,39,19,23,...,305,28,31,109,8,6,1020,84,5,3


#### QUERY 실행을 완료한 후 MYSQL에 최종 적용

In [154]:
conn.commit()